In [1]:
%reload_ext autoreload
%autoreload 2

In [5]:
from utils import *
from nlp import *

In [6]:
import pickle

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


fpath = 'qa_corpus.csv'
content = pd.read_csv(fpath, encoding='utf-8')

In [7]:
ltp = Ltp(seg=True, pos=True, seg_lexicon_path='data/lexicon_seg.txt', pos_lexicon_path='data/lexicon.txt')
text_precess = ProcessText()
stopwords = load_stop(['data/chinese_stopwords.txt', 'data/哈工大停用词表.txt'])

In [2]:
# for idx, row in content.iterrows():
#     if '您好' in row.question:
#         print(row.question, row.answer, idx)

In [4]:
# len(content)

In [15]:
content.to_csv('qa_corpus.csv', encoding='utf-8', index=False)

In [80]:
# 问答对中有的词，保留

# def test_in_questions(string):
#     for i, s in enumerate(content.question):
#         if string == str(s).lower().strip():
# #             print(i, s)
# #             print(content.answer[i])
#             return 1
#     return 0

# out = []
# for i in stopwords:
#     if test_in_questions(i):
#         continue
#     else:
#         out.append(i)

# with open('task_stopwords.txt', 'w', encoding='utf-8') as f:
#     for w in out:
#         f.write(w + '\n')

# stopwords = load_stop(['task_stopwords.txt'])

In [6]:
# test
q_content = text_precess(content.question[1], ltp.segmentor.segment)
print(q_content)

代发 工资


In [7]:
def save_pickle(path, data):
    # pickle不能保存lambda
    fileObject = open(path, 'wb')
    pickle.dump(data, fileObject)
    fileObject.close()

## tf-idf

In [11]:
from functools import partial

f = partial(text_precess, cut_method=ltp.segmentor.segment)

content['q_processed'] = content.question.apply(f)

In [12]:
content.head()

,qid,question,answer,q_processed
0,1,补发网银盾,您可以到智慧柜员机办理此项业务，方便快捷，节省您的宝贵时间哦。若去柜台办理，请刷身份证取号。,补发 网银 盾
1,2,代发工资,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。,代发 工资
2,3,对帐单查询打印,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。,对 帐单 查询 打印
3,4,理财产品取号,请说出您需要办理的业务,理财 产品 取号
4,5,密码修改,请您带上身份证到自助柜员机办理，简单快捷，操作容易哦。,密码 修改


In [13]:
content.to_csv('qa_processed', encoding='utf-8', index=False)

In [121]:
# content_1 = pd.read_csv('qa_processed', encoding='utf-8')
# content_1.head()

,qid,question,answer,q_processed
0,1,补发网银盾,您可以到智慧柜员机办理此项业务，方便快捷，节省您的宝贵时间哦。若去柜台办理，请刷身份证取号。,补发 网银 盾
1,2,代发工资,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。,代发 工资
2,3,对帐单查询打印,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。,对 帐单 查询 打印
3,4,理财产品取号,请说出您需要办理的业务,理财 产品 取号
4,5,密码修改,请您带上身份证到自助柜员机办理，简单快捷，操作容易哦。,密码 修改


In [10]:
# 注意，自定义了analyzer，忽略了stopwords
def split_func(s):
    return s.split()

vectorized = TfidfVectorizer(max_features=12000, tokenizer=split_func, stop_words=stopwords)
X = vectorized.fit_transform(content['q_processed'])

print(len(vectorized.vocabulary_))
print(X.shape)

d:\programdata\envs\nlplearning\lib\site-packages\sklearn\feature_extraction\text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', 'δ', 'ψ', 'в', 'ⅲ', '＜δ', '［①ｂ］', '［③ｆ］', 'ａ', 'ｌｉ', 'ｒ．ｌ．', 'ｚｘｆｉｔｌ'] not in stop_words.
  sorted(inconsistent))


10981
(32526, 10981)


In [11]:
word_2_id = vectorized.vocabulary_

save_pickle('data/tfidf_word2vec.bin', word_2_id)

In [12]:
vectorizer_path = 'model/tfidf.model'

# pickle不能保存lambda
fileObject = open(vectorizer_path, 'wb')
pickle.dump(vectorized, fileObject)
fileObject.close()

In [10]:
from copy import deepcopy

word_2_id = deepcopy(vectorized.vocabulary_)
id_2_word = {d: w for w, d in word_2_id.items()}

In [14]:
len(word_2_id)

10981

In [15]:
# tfidf的词表忽略了一些词，这些词在进行word vec之类的模型时，可以加入使用。Elmo等
for q in content.q_processed:
    for w in q.split():
        if w not in word_2_id:
            # print("adding", w)
            word_2_id[w] = len(word_2_id)
            
print(len(word_2_id))

11320


In [16]:
# 单独保存全量词表
save_pickle('data/full_word2id.bin', word_2_id)
# save_pickle('data/full_id2word.bin', id_2_word)

In [11]:
from operator import and_
from functools import reduce

# 倒排索引召回
inverse_idx = X.transpose()

and_pos_set = {'n', 'v', 'm', 'nh', 'ni', 'nl', 'ns', 'nt', 'ws'}


def boolen_search(doc_word):
    inverse_data = doc_word.transpose()
    inverse_idx = []
    for arr in inverse_data:
        inverse_idx.append(set(np.nonzero(arr.toarray()[0])[0]))
    return inverse_idx
    

inverse_table = boolen_search(X)    

In [90]:
from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)

In [120]:
k_largest_heap = KLargest(10, [])

In [137]:
def search_tfidf(query):
    """"""
    words = text_precess(query, ltp.segmentor.segment)
    cws_list = words.split()
    postags = ltp.postagger.postag(cws_list)


    to_and = []
    for i, postag in enumerate(postags):
        if postag in and_pos_set:
            to_and.append(cws_list[i])
    print(to_and)
    try:
        candidates_ids = [word_2_id[w] for w in to_and]
    except KeyError:
        pass
    
    # 布尔搜索：找到同时包含关键词的文档
    documents_ids = [
         inverse_table[_id] for _id in candidates_ids
    ]
    merged_documents = reduce(and_, documents_ids)
    
    query_vec = vectorized.transform([words]).toarray()
    # 计算相似性排序
    k_largest_heap = KLargest(10, [])
    for i in merged_documents:
        a = PrioritizedItem(-distance(query_vec, X[i].toarray()), i)
        k_largest_heap.add(a)
        
    sorted_docuemtns_id = [i.item for i in sorted(k_largest_heap.pool, reverse=True)]
    
#     sorted_docuemtns_id = sorted(merged_documents, key=lambda i: distance(query_vec, X[i].toarray()))
#     a = [distance(query_vec, X[i].toarray()) for i in sorted_docuemtns_id]
    
    return sorted_docuemtns_id

In [138]:
search_tfidf('办理龙卡')

['办理', '龙卡']


[32202, 344, 30939, 32362, 30934, 20038, 30928, 30914, 30938, 31033]

In [119]:
search_tfidf('办理龙卡')

['办理', '龙卡']


([32202,
  344,
  30939,
  32362,
  30934,
  20038,
  30928,
  30914,
  30938,
  31033,
  21590,
  32090,
  20446,
  22636,
  20035,
  20563,
  268,
  20942,
  22639,
  22604,
  20617,
  21122,
  21006,
  22722,
  22721,
  20460,
  21511,
  30941,
  32363,
  22368,
  20023,
  22119,
  20570,
  22635,
  23133,
  22640,
  20390,
  22365],
 [0.341292547087244,
  0.3538401266163863,
  0.45032045652448693,
  0.45032045652448693,
  0.45448519447742286,
  0.4921195827671576,
  0.4934197202013858,
  0.49372203293686745,
  0.5100788000908579,
  0.5122536771422572,
  0.5174639945858079,
  0.5371732140374923,
  0.5461049088807657,
  0.5585875607087916,
  0.5668534532242864,
  0.5673348366554356,
  0.5683702898522947,
  0.5743790597252916,
  0.5775382465273065,
  0.580778903873755,
  0.5837367571160648,
  0.5856567300619067,
  0.5905027373441605,
  0.6095259170543914,
  0.6176184718625879,
  0.6277104617783331,
  0.6425604284835089,
  0.6430452942959818,
  0.6430452942959818,
  0.6460643459286424,

In [20]:
save_pickle('data/doc_tfidf_vec.bin', X)

In [21]:
save_pickle('data/inverse_idx_table.bin', inverse_table)

## BM25

In [22]:
from collections import Counter
from math import log

def get_tf(sent_i, sent_j):
    """计算bm25的term frequence. sent来自预处理的sent_of_words列表。"""
    freq = {}
    sent_i_counts = Counter(sent_i)
    # 计算i句中的词，在j句中的tf
    for w in sent_j:
        # if not self.filte_words(w_item):
        #     continue
        if w in sent_i_counts:
            freq[w] = sent_i_counts[w]
        else:
            freq[w] = 0
    total = len(sent_i)
    return {word: count / total for word, count in freq.items()}


def get_idf(content):
    """计算inverse document frequence. 这里计算句子的相似度，所以计算inverse sentence frequence"""
    total_sent = len(content) + 1 # 假设有一个句子包含所有词
    avg_len = 0
    doc_freq = {}
    for sent in content:
        sent = sent.split()
        avg_len += len(sent)
        words = list({w for w in sent})
        for word in words:
            # 假设有一个句子包含所有词
            count = doc_freq.setdefault(word, 1)
            doc_freq[word] = count + 1
    avg_len /= total_sent
    # sklearn中的实现方式
    idf = {word: log(total_sent / df) + 1 for word, df in doc_freq.items()}
    return idf, avg_len


def sent_corelation_func(sent_i, sent_j, idf, avg_len, k1=1.5, b=0.75):
    """计算bm25。

    sent_i ： 与query对比的句子，在文档中进行遍历，D
    sent_j : query的句子, Q
    """
    tf = get_tf(sent_i, sent_j)

    K = k1 * (1 - b + b * len(sent_i) / avg_len)
    bm25 = 0
    for j_word in sent_j:
        bm25 += idf[j_word] * tf[j_word] * (k1 + 1) / (tf[j_word] + K)
    return bm25

In [23]:
idf, avg_len = get_idf(content.q_processed)

In [25]:
save_pickle('data/idf_and_avglen.bin', (idf, avg_len))

In [26]:
sent_corelation_func(content.q_processed[21231].lower().split(), content.q_processed[21230].lower().split(), idf, avg_len)

8.148760719175181

In [27]:
for i, j in content[['qid', 'question']].iterrows():
    print(j)
    break

qid             1
question    补发网银盾
Name: 0, dtype: object


## 词向量模型

In [ ]:
from gensim.models.fasttext import load_facebook_model


fb_model  = load_facebook_model('cc.zh.300.bin')

new_sent = list(map(lambda s: s.lower().split(), content.q_processed))
len(new_sent)

fb_model.build_vocab(new_sent, update=True)
fb_model.train(sentences=new_sent, total_examples=len(new_sent), epochs=5)

In [ ]:
selected = []
for w in word_2_id.keys():
    vec = fb_model.wv[w]
    selected.append((w, vec))

In [ ]:
with open('selected_words_fasttext_300.txt', 'w', encoding='utf-8') as f:
    first_line = str(len(selected)) + '\t' + str(300)
    f.write(first_line)
    for w, v in selected:
        line =  '\n' + str(w) + '\t' + array_to_string(v)
        f.write(line)

In [40]:
# 上次处理，保存格式不对，所以再处理。。
with open('selected_words_fasttext_300.txt', 'r', encoding='utf-8') as f:
    with open('wordvec_fasttext_300.txt', 'w', encoding='utf-8') as ff:
        file = f.readlines()
        ff.write(file[0].replace('\t', ' '))
        for line in file[1: ]:
            ff.write(line.replace('\t', ' '))

In [41]:
from gensim.models import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('wordvec_fasttext_300.txt')

### 索引

当然可以使用kmeans之类的聚类方法，但是此处利用annoy（基于树的向量聚类）进行稠密向量的索引和聚类

In [42]:
from annoy import AnnoyIndex

In [91]:
dim = 300

index = AnnoyIndex(dim, 'angular')

In [95]:
sent_avg_vec(content.q_processed[2], word_vectors).shape

(300,)

In [85]:
from operator import or_


def search_sentvec(query):
    """"""
    words = text_precess(query, ltp.segmentor.segment)
    cws_list = words.split()
    postags = ltp.postagger.postag(cws_list)

    to_and = []
    for i, postag in enumerate(postags):
        if postag in and_pos_set:
            to_and.append(cws_list[i])
    print(to_and)
    
    try:
        candidates_ids = [word_2_id[w] for w in to_and]
    except KeyError:
        pass

    documents_ids = [
         set(np.nonzero(inverse_idx[_id].toarray()[0])[0]) for _id in candidates_ids
    ]

    merged_documents = reduce(or_, documents_ids)
    sorted_docuemtns_id = sorted(merged_documents, 
                                 key=lambda i: distance(sent_avg_vec(cws_list, word_vectors), 
                                                                          sent_avg_vec(content.q_processed[i].split(), word_vectors)))

    return sorted_docuemtns_id

In [86]:
search_sentvec('办理龙卡')

['办理', '龙卡']


[344,
 21590,
 268,
 20460,
 20038,
 30928,
 265,
 22188,
 30939,
 32362,
 273,
 30914,
 20563,
 30934,
 20945,
 757,
 20942,
 272,
 21511,
 21006,
 21132,
 22331,
 20489,
 392,
 61,
 20446,
 20941,
 20035,
 269,
 23101,
 21768,
 19945,
 21457,
 267,
 274,
 22721,
 933,
 20617,
 22604,
 22636,
 22717,
 22639,
 22290,
 20034,
 346,
 345,
 32090,
 32202,
 20396,
 22453,
 22629,
 31033,
 22313,
 22722,
 969,
 22273,
 452,
 30938,
 21882,
 19940,
 21076,
 42,
 21099,
 20365,
 24636,
 20337,
 32299,
 21336,
 20805,
 22452,
 31341,
 21617,
 13326,
 16106,
 21335,
 22244,
 20348,
 20565,
 21122,
 167,
 30941,
 32363,
 22745,
 22635,
 22393,
 21082,
 20142,
 30862,
 21085,
 23207,
 20812,
 31257,
 30908,
 23092,
 19989,
 22642,
 20934,
 22119,
 22291,
 20143,
 23721,
 19947,
 20428,
 20912,
 19810,
 20940,
 22702,
 20569,
 22444,
 20573,
 20804,
 20134,
 22120,
 20803,
 19990,
 21173,
 32332,
 22640,
 21106,
 20366,
 266,
 21052,
 19960,
 21842,
 22451,
 21337,
 20140,
 23410,
 22243,
 20493,


In [84]:
search_sentvec('办理龙卡')

['办理', '龙卡']


[344,
 21590,
 268,
 20460,
 20038,
 30928,
 265,
 22188,
 30939,
 32362,
 273,
 30914,
 20563,
 30934,
 20945,
 757,
 20942,
 272,
 21511,
 21006,
 21132,
 22331,
 20489,
 392,
 61,
 20446,
 20941,
 20035,
 269,
 23101,
 21768,
 19945,
 21457,
 267,
 274,
 22721,
 933,
 20617,
 22604,
 22636,
 22717,
 22639,
 22290,
 20034,
 346,
 345,
 32090,
 32202,
 20396,
 22453,
 22629,
 31033,
 22313,
 22722,
 969,
 22273,
 452,
 30938,
 21882,
 19940,
 21076,
 42,
 21099,
 20365,
 24636,
 20337,
 32299,
 21336,
 20805,
 22452,
 31341,
 21617,
 13326,
 16106,
 21335,
 22244,
 20348,
 20565,
 21122,
 167,
 30941,
 32363,
 22745,
 22635,
 22393,
 21082,
 20142,
 30862,
 21085,
 23207,
 20812,
 31257,
 30908,
 23092,
 19989,
 22642,
 20934,
 22119,
 22291,
 20143,
 23721,
 19947,
 20428,
 20912,
 19810,
 20940,
 22702,
 20569,
 22444,
 20573,
 20804,
 20134,
 22120,
 20803,
 19990,
 21173,
 32332,
 22640,
 21106,
 20366,
 266,
 21052,
 19960,
 21842,
 22451,
 21337,
 20140,
 23410,
 22243,
 20493,


In [74]:
search_sentvec('办理信用卡')

['办理', '信用卡']


[969,
 13326,
 16106,
 32299,
 22745,
 452,
 30844,
 20428,
 244,
 31957,
 580,
 20934,
 20565,
 31995,
 22393,
 24636,
 31964,
 32051,
 20551,
 32332,
 31802,
 32397,
 21024,
 1025,
 624,
 24496,
 22783,
 508,
 19909,
 30822,
 20429,
 31687,
 23199,
 770,
 21619,
 858,
 19892,
 11984,
 20036,
 32064,
 20029,
 32357,
 23466,
 20502,
 32226,
 32202,
 30928,
 31033,
 23761,
 21507,
 32180,
 23960,
 32090,
 24355,
 23695,
 32029,
 20433,
 32245,
 21151,
 32052,
 30785,
 20092,
 20600,
 20084,
 32033,
 24295,
 19931,
 23114,
 19975,
 30938,
 21175,
 406,
 31460,
 31668,
 24076,
 553,
 24269,
 22383,
 32278,
 208,
 24829,
 22111,
 20528,
 22112,
 20446,
 21134,
 32070,
 21521,
 20810,
 23110,
 26638,
 31626,
 31677,
 25685,
 21100,
 21102,
 23760,
 21932,
 22946,
 23804,
 25540,
 25917,
 31305,
 20763,
 20733,
 453,
 21103,
 23896,
 344,
 21924,
 23833,
 21179,
 25610,
 32062,
 21923,
 25551,
 21590,
 24082,
 20035,
 21572,
 19994,
 20796,
 28651,
 25723,
 20549,
 32272,
 21925,
 22477,
 30

In [58]:
def show_content(idx):
    print(content.iloc[idx])

In [59]:
show_content(344)

qid                                                          461
question                                                 办理陆港通龙卡
answer         客户申请陆港通龙卡，须本人前往柜台填写《陆港通龙卡服务申请表格》，同时须提供有效身份证件和地...
q_processed                                            办理 陆港通 龙卡
Name: 344, dtype: object


In [82]:
show_content(20460)

qid                                               51456
question                             “华东师大龙卡名校卡”可否办理附属卡
answer         “华东师大龙卡名校卡”有附属卡（学生身份持卡人除外），附属卡卡面采用标准卡卡面。
q_processed                       华东师大 龙卡 名校 卡可否 办理 附属卡
Name: 20460, dtype: object


In [105]:
vec_list = get_wordvec_dict(content.q_processed, word_vectors)

In [ ]:
len(vec_list)

In [100]:
def build_annoy(n_trees=10):
    for i, vec in enumerate(vec_list):
        index.add_item(i, vec)
    index.build(n_trees)

In [106]:
build_annoy()

In [108]:
# test
q = sent_avg_vec('办理 龙卡', word_vectors)

index.get_nns_by_vector(q, 20, search_k=-1, include_distances=True)

([344,
  21590,
  268,
  20460,
  20038,
  30928,
  265,
  22188,
  30939,
  32362,
  273,
  30914,
  20563,
  30934,
  757,
  20945,
  20942,
  272,
  21511,
  21006],
 [0.0634312629699707,
  0.3531995415687561,
  0.406412810087204,
  0.422742635011673,
  0.42773884534835815,
  0.43435052037239075,
  0.4558327794075012,
  0.4581727981567383,
  0.4645462930202484,
  0.4645462930202484,
  0.4649970829486847,
  0.46618008613586426,
  0.4672144651412964,
  0.46789371967315674,
  0.47393569350242615,
  0.47393569350242615,
  0.4779311418533325,
  0.4825846552848816,
  0.48646438121795654,
  0.4867269694805145])

可以看到 annoy 返回结果是和暴力搜索的结果基本一致的。

另一种采用kmeans预处理，划分搜索范围的方法，依赖于算法划分的准确度，一旦在某一个子集中搜索，不会在另一个子集中搜索。因为预处理时，是将kmeans预处理结果作为标签，保存下来的，所以固化了搜索范围。

In [109]:
index.save('data/sentvec.ann')

True

In [110]:
def search_sentvec(query, top_k=20, include_distances=False):
    words = text_precess(query, ltp.segmentor.segment)
    
    q_vec = sent_avg_vec(words, word_vectors)
    
    sorted_docuemtns_id = index.get_nns_by_vector(q_vec, top_k, search_k=-1, include_distances=include_distances)
    
    return sorted_docuemtns_id

In [107]:


# 直接用库
import heapq
class KthLargest_2:
    def __init__(self, k: int, nums):
        self.pool = heapq.nlargest(k, nums)
        heapq.heapify(self.pool)
        self.k = k

    def add(self, val: int) -> int:
        if len(self.pool) < self.k:
            heapq.heappush(self.pool,val)
        else:
            heapq.heappushpop(self.pool,val)
        return self.pool[0]


# Your KthLargest object will be instantiated and called as such:
if __name__ == "__main__":
    k = 2
    nums = []
    obj = KthLargest_2(k, nums)
    for i in range(10):
        a = PrioritizedItem(i, 'a')
        param_1 = obj.add(a)
        print(param_1)
        print(obj.pool)


PrioritizedItem(priority=0, item='a')
[PrioritizedItem(priority=0, item='a')]
PrioritizedItem(priority=0, item='a')
[PrioritizedItem(priority=0, item='a'), PrioritizedItem(priority=1, item='a')]
PrioritizedItem(priority=1, item='a')
[PrioritizedItem(priority=1, item='a'), PrioritizedItem(priority=2, item='a')]
PrioritizedItem(priority=2, item='a')
[PrioritizedItem(priority=2, item='a'), PrioritizedItem(priority=3, item='a')]
PrioritizedItem(priority=3, item='a')
[PrioritizedItem(priority=3, item='a'), PrioritizedItem(priority=4, item='a')]
PrioritizedItem(priority=4, item='a')
[PrioritizedItem(priority=4, item='a'), PrioritizedItem(priority=5, item='a')]
PrioritizedItem(priority=5, item='a')
[PrioritizedItem(priority=5, item='a'), PrioritizedItem(priority=6, item='a')]
PrioritizedItem(priority=6, item='a')
[PrioritizedItem(priority=6, item='a'), PrioritizedItem(priority=7, item='a')]
PrioritizedItem(priority=7, item='a')
[PrioritizedItem(priority=7, item='a'), PrioritizedItem(priority=

## Bert

In [1]:
from bert_serving.client import BertClient

In [2]:
bc = BertClient()

In [4]:
bc.encode(['你好', '你好啊', '早上好'])

array([[ 0.6709697 , -0.11540446,  0.49079838, ...,  0.25540185,
         0.57633656, -0.05261125],
       [ 0.6727595 , -0.22872677,  0.34226033, ...,  0.305778  ,
         0.35676366, -0.19390674],
       [ 0.4310699 ,  0.11017738,  0.13510828, ..., -0.20400193,
         0.0228704 ,  0.5595023 ]], dtype=float32)

In [5]:
list(zip(*[(1, '1'), (2, "2")]))

[(1, 2), ('1', '2')]

## Eval

In [12]:
import pandas as pd

In [13]:
data = pd.read_csv('./data/eval_data.csv')

In [15]:
def check_manul(data):
    score = 0
    for row in data.iterrows():
        print(row)
        v = int(input('对或错：'))
        score += v
    return score / 100

In [ ]:
check_manul(data)

(0, Unnamed: 0             0
top0          我记忆力不好拉~呵呵
top1              别老爱撒娇啊
top2                你识字吗
top3                学口语。
top4               你有妈妈吗
top5             梳头为啥能健脑
top6          赞美是一种说话的艺术
top7               你妈妈是谁
top8              说话要有礼貌
top9                  健谈
input         我记忆力不好拉~呵呵
model           my_model
Name: 0, dtype: object)
对或错：1
(1, Unnamed: 0               1
top0          下雪的原理是什么你知道吗
top1          下雨的原理是什么你知道吗
top2            你知道为什么会下雪吗
top3            你知道为什么会下雨吗
top4               你是啥都知道吗
top5           今天的天气如何你知道吗
top6           你知道今天是什么日子吗
top7               你知道支付宝吗
top8                  我知道啊
top9            今天有什么事情要做的
input         下雪的原理是什么你知道吗
model             my_model
Name: 1, dtype: object)
对或错：1
(2, Unnamed: 0                              2
top0                           龙支付可以在哪里使用
top1                            如何使用龙支付付款
top2                        如何使用龙支付“收款”功能
top3                       如何使用龙支付“向朋友付款”
top4                       如何在善融商城使用龙支付购物
